# Esercizio 5 - Automatic summarization

Si vuole ridurre un documento del 10%, 20% o 30% secondo la seguente strategia

1. Individuare l'argomento del testo che si sta riassumendo; l'argomento può essere indicato come un (insieme di) vettori NASARI:
    - vt1 = {term1_score, term2_score, …, term10_score }
    - vt2 = {term1_score, term2_score, …, term10_score } 
    - ...

2. creare il contesto, raccogliendo qui i vettori dei termini (questo passaggio può essere ripetuto, scaricando ad ogni round il contributo dei termini associati)

3. conservare i paragrafi le cui frasi contengono i termini più salienti, in base alla sovrapposizione ponderata, WO(v1,v2)
    - riclassificare il peso dei paragrafi applicando almeno uno degli approcci menzionati (titolo, spunto, frase, coesione).

In [31]:
import requests
import os
import pandas as pd
import re
import math
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import MWETokenizer #tiene conto delle multiword expressions
from nltk.corpus import wordnet as wn
import json

BABELNET_TOKEN = '1e258739-f5e4-4961-8267-a2da4fe94572' #MO
#BABELNET_TOKEN = '01a5d861-2f36-45cb-8974-a2a6526530d2' #LT

## Pre-processing

Metodo utilizzato per eseguire il preprocessing delle frasi, in cui vengono effettuate le seguenti operazioni:
- Rimozione della punteggiatura
- Trasformazione delle lettere in lowercase
- Tokenizzazione della frase tenendo conte delle multiword expression
- Lemmatizzazione di tutte le parole
- Rimozione delle stop words

In [17]:
stop_words = set(stopwords.words('english')) #remove stop words
mwes = [x for x in wn.all_lemma_names() if '_' in x]
mwes = [tuple(x.split('_')) for x in mwes]
tokenizer = MWETokenizer(mwes, separator=' ')
lemmatizer = WordNetLemmatizer()

def pre_processing(document):
    document = re.sub(r'[^\w\s]',' ',document) #remove punctuation
    document = document.lower()
    document = tokenizer.tokenize(document.split())
    document = [lemmatizer.lemmatize(token) for token in document]  
    document = [w for w in document if not w in stop_words]
    return document

## Babelnet Id di una frase

Viene utilizzato principalmente per ottenere i Babelnet Id delle parole del titolo, che una volta sottosposti a WSD (in quanto, molto probabilmente, per ogni parola avremo più synset) ci serviranno per ottenere i vettori Nasari

In [25]:
def get_babelnet_synset_by_word(word):
    return requests.get(f'https://babelnet.io/v8/getSynsetIds?lemma={word}&searchLang=EN&key={BABELNET_TOKEN}').json()
'''
Data una frase restituisce tutti i suoi babelnet id
'''
def get_sentence_babelnet_ids(file_name, sentence):
    if os.path.exists('data/ids-'+ file_name +'.json'):
        with open('data/ids-'+ file_name +'.json') as json_file:
            ids = json.load(json_file)
    else:
        ids = {}
        # prendo gli id di babelnet per ogni parola della frase
        for word in sentence:
            ids[word] = requests.get(f'https://babelnet.io/v8/getSynsetIds?lemma={word}&searchLang=EN&key={BABELNET_TOKEN}').json()

        # prendo i synset di babelnet per ogni parola della frase
        with open('data/ids_'+ file_name +'.json', 'w') as fp:
            json.dump(ids, fp)

    return ids

'''
Dato un babelnet id guardo nel file locale se ho già le informazioni, altrimenti 
faccio una richiesta a babelnet e aggiungo la riga al file locale
'''
def get_babelnet_synset_by_id(syn_id):
    df = pd.read_csv('data/local_babelnet_syns.csv')
    glosses = ['']
    examples = ['']

    if syn_id in df['id'].values:
        row = df[df['id'] == syn_id]
        name = row['name'].values[0]
        if not 'nan' in str(row['glosses'].values[0]):
            glosses = row['glosses'].values[0].split(';')
        if not 'nan' in str(row['examples'].values[0]):
            examples = row['examples'].values[0].split(';')             
    else:
        response = requests.get(f'https://babelnet.io/v8/getSynset?id={syn_id}&key={BABELNET_TOKEN}').json()


        print('=============')
        print(response)
        print('=============')

        name = response['senses'][0]['properties']['fullLemma']
        glosses = ""
        for gloss in response['glosses']:
            glosses += str(gloss['gloss']) + ';'
        examples = ""
        for example in response['examples']:
            examples += str(example['example']) + ';'

        #add row to df and save it
        df = df.append({'id': syn_id, 'name': name, 'glosses': glosses, 'examples': examples}, ignore_index=True)
        df.to_csv('data/local_babelnet_syns.csv', index=False)

        glosses = glosses.split(';')
        examples = examples.split(';')

    return syn_id, name, glosses, examples

## Nasari

Estrazione dei vettori Nasari dal file locale

In [19]:
def get_nasari_vectors():
    nasari_vectors = pd.read_csv('data/dd-nasari.txt', on_bad_lines='skip', header=None, sep=';')
    nasari_vectors = nasari_vectors.set_index(0)
    return nasari_vectors

## Algoritmo Simplified Lesk

Mi server per fare il WSD dei synsets ottenuti dei token del titolo

In [20]:
'''
Dati i tutti i synset di una parola e il suo contesto, andando ad applicare
il Lesk, restituisce il synset con il contesto più simile
'''
def get_signature(bn_syn):
    _, _, glosses, examples = get_babelnet_synset_by_id(bn_syn)

    signature = ""
    for gloss in glosses:
        signature += gloss + ' '
    for example in examples:
        signature += example + ' '
    return set(pre_processing(signature))

# Usa come contesto l'intero testo del file, non solo il titolo
def simplified_lesk(bn_syns, context):
    best_sense = bn_syns[0]['id']
    max_overlap = 0
    
    for bn_syn in bn_syns:
        signature = get_signature(bn_syn['id'])
        overlap = len(context.intersection(signature))
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = bn_syn['id']
    
    return best_sense

def get_best_senses(ids, context):
    senses = {}
    for word in ids:
        senses[word] = simplified_lesk(ids[word], context)

    return senses

## Estrazione del documento da tradurre

In [21]:
def open_text(file_name):
    text = open('data/docs/' + file_name + '.txt', 'r', encoding='utf-8').read().split('\n')

    # pre processing
    text = [line for line in text if line != '']
    text_preprocessed = [pre_processing(line) for line in text[0:]]

    # prendo il contesto per fare WSD dei babelnet id delle parole del testo
    context_for_wsd = []
    for sentence in text_preprocessed:
        context_for_wsd = context_for_wsd + sentence
    context_for_wsd = set(context_for_wsd)

    # prendo il titolo
    title_preprocessed = text_preprocessed[0]

    return title_preprocessed, text_preprocessed, text, context_for_wsd

# Associazione Vectors - Synsets

Associazione dei vettori Nasari ai Synset disambiguati delle parole del testo

In [22]:
def get_nasari_vectors_by_senses(senses):
    nasari_vectors = get_nasari_vectors()
    vectors = {}

    for word in senses:
        if senses[word] in list(nasari_vectors.index.values):
            #print(f'{senses[word]} in nasari_vectors') 
            vectors[word] = nasari_vectors.loc[senses[word]]
        #else:
            #print(f'{senses[word]} not in nasari_vectors')

    return vectors

In [ ]:
def get_nasari_vector_by_sense(sense):
    nasari_vectors = get_nasari_vectors()
    vector = None
    if sense in list(nasari_vectors.index.values):
        vector = nasari_vectors.loc[sense]
    return vector

## Creazione del contesto per fare text summarization

In [23]:
def get_weighted_context(vectors):
    get_weighted_context = []

    for vector in vectors:
        #print(vectors[vector])
        sum_weights = 0
        for word in vectors[vector]:
            array = word.split('_')
            if len(array) > 1:
                sum_weights += float(word.split('_')[1])
            
        for word in vectors[vector]:
            array = word.split('_')
            if len(array) > 1:    
                weight = float(word.split('_')[1]) / sum_weights
                # preprocess word
                word_to_added = word.split('_')[0]
                word_to_added = re.sub(r'[^\w\s]',' ',word_to_added) #remove punctuation
                word_to_added = word_to_added.lower()
                word_to_added = lemmatizer.lemmatize(word_to_added)

                get_weighted_context.append((word_to_added, weight))

    return get_weighted_context

## Pesatura dei paragrafi

Eseguita usando la metrica Weighted Overlap

In [ ]:
# weighted overlap
def weighted_overlap2(vect1, vect2):
    tot = 0.0
    overlap = 0
    for i, elem in enumerate(vect1):
        try:
            index = vect2.index(elem) + 1
            overlap += 1
        except:
            index = -1
        if index != -1:
            tot += (i + 1 + index) ** (-1)
    denominatore = 1.0
    for i in range(1, overlap + 1):
        denominatore += (2 * i) ** (-1)
    return tot / denominatore

In [ ]:
def disambiguate_word_context(weighted_context):
    synsets_topic = []
    total_weight = 0
    for (word_topic, weight) in weighted_context:
        synsets_topic.append((simplified_lesk(get_babelnet_synset_by_word(word_topic), set([tupla[0] for tupla in weighted_context])), weight))
        total_weight += weight
    return synsets_topic, total_weight

In [ ]:
def compute_score_sentence(sentence, synsets_context, total_weight):
    synsets_sentence = []
    score_sentence = 0
    for word in sentence:
        synsets_sentence.append(simplified_lesk(get_babelnet_synset_by_word(word), set([tupla[0] for tupla in synsets_context])))
    for syn_word in synsets_sentence:
        score_word = 0
        for (syn_topic,weight) in synsets_context:
            score_word += (weighted_overlap2(syn_word, syn_topic)*weight)
        score_word /= total_weight #media ponderata
        score_sentence += score_word
    score_sentence /= len(sentence)
    return score_sentence

In [ ]:
def weighted_overlap(sentence, weighted_context):
    numeratore = 0
    for word in sentence:
        #check if word is in first column of key_words
        if word in [x[0] for x in weighted_context]:
            #print(f'{word} in key_words')
            #get index of word in key_words
            index = [x[1] for x in weighted_context if x[0] == word][0]
            #print(f'index: {index}')

            numeratore += 1/(index)
            #print('\n')
       # else:
            #print(f'{word} not in key_words')
            #print('\n')

    i = 1
    denominatore = 0
    for word in weighted_context:
        denominatore += 1/(2*i)
        i += 1

    return numeratore/denominatore

## Estrazione automatica del riassunto

In [ ]:
def make_summarization(text, text_preprocessed, synsets_context, total_weight, perc=0.8):
    text_preprocessed = text_preprocessed[1:]

    title = text[0]
    text = text[1:]

    weight_sentences = []
    i = 0
    for line in text_preprocessed:
        # attribuisce un peso ad ogni frase
        weight_sentences.append((i, text[i], compute_score_sentence(line, synsets_context, total_weight)))
        i += 1

    # ordina le frasi in base al peso
    weight_sentences = sorted(weight_sentences, key=lambda tup: tup[2], reverse=True)
    # prendi il primo 80% delle weight_sentences
    weight_sentences = weight_sentences[:round(len(weight_sentences) * perc)]
    # ordina le frasi in base all'id
    weight_sentences = sorted(weight_sentences, key=lambda tup: tup[0])

    # prendi solo le frasi
    summary = [x[1] for x in weight_sentences]
    #add in the first postizion the title
    summary.insert(0, title)
    return summary

def print_summary(summary):
    summary = '\n\n'.join(summary)
    print(summary)
    return 

## Main

In [32]:
file_name = 'Andy-Warhol'

In [33]:
title_preprocessed, text_preprocessed, text, context_for_wsd = open_text(file_name)
bn_ids = get_sentence_babelnet_ids(file_name, title_preprocessed)
title_sense = get_best_senses(bn_ids, context_for_wsd)

In [34]:
title_nasari_vectors = get_nasari_vectors_by_senses(title_sense)

FileNotFoundError: [Errno 2] No such file or directory: 'data/dd-nasari.txt'

In [ ]:
weighted_context = get_weighted_context(title_nasari_vectors)

In [ ]:
synsets_context, total_weight = disambiguate_word_context(weighted_context=weighted_context)

In [ ]:
summary = make_summarization(text, text_preprocessed, synsets_context, total_weight)

## Valutazione

la valutazione può essere eseguita sulla base di due metriche complementari
- BLEU (bilingual evaluation understudy) per quanto riguarda la precision
- ROUGE (Recall-Oriented Understudy for Gisting Evaluation) per quanto riguarda la recall

### BLEU

funzione di scoring che è stata elaborata per valutare i sistemi per la traduzione automatica
- costruire un sommario di riferimento, come un elenco di termini rilevanti che dovrebbero essere presenti.
- confrontare l'insieme di termini nel riepilogo automatico (che chiamiamo riepilogo del candidato) con quelli nel riepilogo del candidato.
- il punteggio BLEU è calcolato come P = m/wt che è la frazione di termini del candidato che si trovano nel riferimento, dove m è il numero di termini del candidato che sono nel riferimento, e wt è la dimensione di il candidato

La precision in IR è solitamente definita come 
precision = |{relevant documents} ∩ {retrieved documents}| / |{retrieved documents}|


In [ ]:
file_name_manual_sum = 'Andy-Warhol-manual-sum'

_, _, manual_sum, _ = open_text(file_name_manual_sum)
automatic_sum = make_summarization(text, text_preprocessed, weighted_context)

count = 0
for line in automatic_sum:
    if (line in manual_sum):
        count += 1

print(f'Precision: {count/len(automatic_sum)}')

### RECALL

Questa metrica stima in che misura le parole (e/o n-grammi) nei riassunti di riferimento umano sono apparse nei riassunti creati dal sistema
- ROUGE-N: Sovrapposizione di N-grammi tra candidato e riferimento
riepilogo.
- ROUGE-1 si riferisce alla sovrapposizione di unigramma (ogni parola) tra il sommari di sistema e di riferimento.

La Recall in IR è abitualmente definito come recall =|{relevant documents} ∩ {retrieved documents}| / |{relevant documents}|



In [ ]:
file_name_manual_sum = 'Andy-Warhol-manual-sum'

_, _, manual_sum, _ = open_text(file_name_manual_sum)
automatic_sum = make_summarization(text, text_preprocessed, weighted_context)

count = 0
for line in automatic_sum:
    if (line in manual_sum):
        count += 1

print(f'Recall: {count/len(manual_sum)}')